## Evaluating mid size deletions

This notebook evaluates the results of the Long Ranger mid size deletion calls for NA12878.

This notebook will not function as-is. In order to get it to run, you will need to have the [Comparative Annotation Toolkit](https://github.com/ComparativeGenomicsToolkit/Comparative-Annotation-Toolkit) installed to get access to the `tools` library. You will also need the SV analysis tool SURVIVOR, and access to Long Ranger VCFs as well as the Personalis truth set VCF.

In [1]:
from tools.fileOps import *
from tools.procOps import *
import vcf
import os
from collections import *
import pandas as pd
import numpy as np
import sys
import os
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42

def close_plot(pdf):
    """Convenience wrapper for producing PDFs"""
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)
    pdf.savefig(bbox_inches='tight')
    plt.close('all')

In [1]:
# PLACE YOUR LONG RANGER VCFS HERE.
vcfs = []

In [73]:
# combine all VCFs and filter for passing deletions >50bp and <30kb
c = Counter()  # keep track of provenance
min_size = 50
max_size = 30000
def parse_gatk(f):
    recs = [x for x in vcf.Reader(open(f)) if x.is_deletion and len(x.FILTER) == 0]
    tmp_sizes = []
    for rec in recs:
        start, stop = rec._compute_coordinates_for_indel()
        tmp_sizes.append(stop - start)
        rec.INFO['SVLEN'] = stop - start
        rec.INFO['SVTYPE'] = 'DEL'
    return recs, tmp_sizes

def parse_lr(f):
    recs = [x for x in vcf.Reader(open(f)) if x.INFO['SVTYPE'] == 'DEL' and len(x.FILTER) == 0]
    for x in recs:
        x.INFO['SVLEN'] = abs(x.INFO['SVLEN'])
    tmp_sizes = [x.INFO['SVLEN'] for x in recs]
    return recs, tmp_sizes

for f in vcfs:
    fn = parse_gatk if 'phased_variants' in f else parse_lr
    recs, tmp_sizes = fn(f)
    filt = [[rec, s] for rec, s in zip(recs, tmp_sizes) if min_size <= s <= max_size]
    t = os.path.basename(f).split('.')[0]
    c[t] = len(filt_recs)
    with open(t + '.filt.vcf', 'w') as outf:
        outf_handle = vcf.Writer(outf, vcf.Reader(f))
        for rec, s in filt:
            outf_handle.write_record(rec)
        outf_handle.close()
    avg = np.nanmean([x[1] for x in filt])
    num_het = len([x for x in filt if x[0].samples[0].is_het])
    num_hom = len(filt) - num_het
    print '{}: avg len {} num het: {} num_hom: {}'.format(t, avg, num_het, num_hom)

dels: avg len 848.530840214 num het: 2393 num_hom: 1725


/mnt/home/ian.fiddes/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: Mean of empty slice


large_svs: avg len nan num het: 0 num_hom: 0
phased_variants: avg len 79.132127193 num het: 1135 num_hom: 689


In [49]:
# large SV file is empty
!printf 'dels.filt.vcf\nphased_variants.filt.vcf\n' > fofn_tmp
!SURVIVOR merge fofn_tmp 100 1 1 1 0 50 survivor_merged.vcf
!rm fofn_tmp

merging entries: 4118
merging entries: 1824


In [177]:
# parse SURVIVOR results

def convert_call(call):
    call = call.replace('|', '/')
    if call == '0/1' or call == '1/0':
        return 'het'
    else:
        return 'hom'

results = []
for l in open('survivor_merged.vcf'):
    if not l.startswith('#'):
        l = l.rstrip().split('\t')
        d = dict(x.split('=') for x in l[7].split(';') if x)
        truth_supp, lr_supp = map(bool, map(int, list(d['SUPP_VEC'])))
        dels_gt = convert_call(l[-2].split(':')[0])
        gatk_gt = convert_call(l[-1].split(':')[0])
        results.append([l[0], l[1], truth_supp, lr_supp, float(d['AVGLEN']), dels_gt, gatk_gt])
survivor_df = pd.DataFrame(results, columns=['chrom', 'pos', 'lr_supp', 'gatk_supp', 'svlen', 'dels_gt', 'gatk_gt'])

In [178]:
survivor_df.svlen.mean()

696.2259540498443

In [179]:
c = Counter()
for _, s in survivor_df.iterrows():
    if s.lr_supp and s.gatk_supp:
        if s.dels_gt == s.gatk_gt:
            c['agree_{}'.format(s.dels_gt)] += 1
        else:
            c['disagree_{}_{}'.format(s.dels_gt, s.gatk_gt)] += 1
    elif s.lr_supp:
        c[s.dels_gt] += 1
    else:
        c[s.gatk_gt] += 1
for x, y in sorted(c.iteritems(), key=lambda x: -x[1]):
    print x, y

het 2685
hom 1829
agree_het 330
agree_hom 209
disagree_het_hom 44
disagree_hom_het 39


In [51]:
# plot the results
with open('gatk_lr_size_distribution_shared.pdf', 'w') as outf, PdfPages(outf) as pdf:
    fig, ax = plt.subplots()
    both = map(int, map(round, map(float, list(survivor_df[(survivor_df.gatk_supp == True) & (survivor_df.lr_supp == True)].svlen))))
    gatk_only = map(int, map(round, map(float, list(survivor_df[(survivor_df.gatk_supp == True) & (survivor_df.lr_supp == False)].svlen))))
    lr_only = map(int, map(round, map(float, list(survivor_df[(survivor_df.gatk_supp == False) & (survivor_df.lr_supp == True)].svlen))))
    cutoff = 1000
    both = [x for x in both if x <= cutoff]
    lr_only = [x for x in lr_only if x <= cutoff]
    gatk_only = [x for x in gatk_only if x <= cutoff]
    ax.hist([both, gatk_only, lr_only], 200, histtype='stepfilled', label=['shared', 'GATK', 'dels'], alpha=0.4)
    ax.set_xlabel('Size of deletion')
    ax.set_ylabel('Number of deletion calls')
    ax.set_xlim((50, 1000))
    ax.legend()
    fig.suptitle('SURVIVOR merging of GATK and LongRanger deletions in 50-1000bp range')
    close_plot(pdf)

In [55]:
r = Counter()
for _, s in survivor_df.iterrows():
    if s.lr_supp and s.gatk_supp:
        r['shared'] += 1
    elif s.lr_supp:
        r['dels'] += 1
    else:
        r['gatk'] += 1
print r

Counter({'dels': 3445, 'gatk': 1069, 'shared': 622})


In [152]:
# try with Personalis/svclassify VCF
# can be downloaded here also:
# ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/technical/svclassify_Manuscript/Supplementary_Information/svclassify/svviz/input_vcf/
vcfs = []
with TemporaryDirectoryPath() as tmp_dir:
    for v in vcfs:
        !sed 's/ /\t/g' {v} > {os.path.join(tmp_dir, os.path.basename(v))}
    vcfs = glob(os.path.join(tmp_dir, '*'))
    !vcf-concat {' '.join(vcfs)} > Personalis.hs37d5.vcf

In [155]:
with open('Personalis.hs37d5.filtered.vcf', 'w') as outf:
    for l in open('Personalis.hs37d5.vcf'):
        if l.startswith('#'):
            outf.write(l)
        else:
            l = l.split('\t')
            l[0] = 'chr' + l[0]
            d = dict(x.split('=') for x in l[-1].split(';'))
            if 50 <= int(d['END']) - int(l[1]) <= 30000:
                outf.write('\t'.join(l))

In [156]:
!printf 'Personalis.hs37d5.filtered.vcf\nsurvivor_merged.vcf\n' > fofn_tmp
!SURVIVOR merge fofn_tmp 100 1 1 1 0 50 svclassify_merged_dels_personalis.vcf
!rm fofn_tmp

merging entries: 2294
merging entries: 5136


In [157]:
# load the results
results = []
for l in open('svclassify_merged_dels_personalis.vcf'):
    if not l.startswith('#'):
        l = l.rstrip().split('\t')
        d = dict(x.split('=') for x in l[7].split(';') if x)
        truth_supp, lr_supp = map(bool, map(int, list(d['SUPP_VEC'])))
        results.append([l[0], l[1], truth_supp, lr_supp, d['AVGLEN']])
survivor_df = pd.DataFrame(results, columns=['chrom', 'pos', 'truth_supp', 'lr_supp', 'svlen'])
both = map(int, map(round, map(float, list(survivor_df[(survivor_df.truth_supp == True) & (survivor_df.lr_supp == True)].svlen))))
truth_only = map(int, map(round, map(float, list(survivor_df[(survivor_df.truth_supp == True) & (survivor_df.lr_supp == False)].svlen))))
lr_only = map(int, map(round, map(float, list(survivor_df[(survivor_df.truth_supp == False) & (survivor_df.lr_supp == True)].svlen))))
print len(both), len(truth_only), len(lr_only)

# construct a triplet of plots showing 50-500bp and 500-10000
with open('lr_truth_svclassify_size_distributions_personalis.pdf', 'w') as outf, PdfPages(outf) as pdf:
    fig, axes = plt.subplots(ncols=2)
    # 50-500bp
    ax = axes[0]
    ax.hist([[x for x in both if x <= 500], 
             [x for x in lr_only if x <= 500], 
             [x for x in truth_only if x <= 500]], 
            90, histtype='stepfilled', 
        label=['Clustered', 'LongRanger', 'Svclassify'], alpha=0.4)
    ax.set_xlabel('Size of deletion')
    ax.set_ylabel('Number of deletion calls')
    ax.set_title('50-500bp')
    ax.set_xlim((50, 500))
    # 500-5000bp
    ax = axes[1]
    ax.hist([[x for x in both if 500 <= x <= 10000], 
             [x for x in lr_only if 500 <= x <= 10000], 
             [x for x in truth_only if 500 <= x <= 10000]], 
            90, histtype='stepfilled', 
        label=['Clustered', 'LongRanger', 'Svclassify'], alpha=0.4)
    ax.set_xlabel('Size of deletion')
    ax.set_ylabel('Number of deletion calls')
    ax.set_title('500-10000bp')
    ax.set_xlim((500, 10000))
    ax.legend()
    fig.suptitle('Size distributions of SURVIVOR clustering of LongRanger deletions with Svclassify truth set')
    close_plot(pdf)

2024 257 3109


In [ ]:
# try lumpy
bam = longranger_bam  # add this here!
cmd = '''samtools sort -m 8G -@ 12 -O bam -T tmp.sort -n -o namesorted.bam {bam}
'''
run_proc(cmd.format(bam=bam).split())

In [ ]:
cmd = [['samtools', 'view', '-h', 'namesorted.bam'],
      ['samblaster', '--excludeDups', '--addMateTags', '--maxSplitCount', '2', '--minNonOverlap', '20'],
      ['samtools', 'view', '-S', '-b', '-']]
run_proc(cmd, stdout='sample.bam')

In [ ]:
!samtools view -b -F 1294 sample.bam > sample.discordants.unsorted.bam

In [ ]:
%%bash

samtools view -h sample.bam \
    | /mnt/home/ian.fiddes/lumpy-sv/scripts/extractSplitReads_BwaMem -i stdin \
    | samtools view -Sb - \
    > sample.splitters.unsorted.bam


In [184]:
!sambamba sort sample.discordants.unsorted.bam sample.discordants
!sambamba sort sample.splitters.unsorted.bam sample.splitters

In [186]:
run_proc(['lumpyexpress', '-B', 'sample.bam', 
          '-S', 'sample.splitters.unsorted.sorted.bam', '-D', 
          'sample.discordants.unsorted.sorted.bam', '-o', 'sample.vcf'])

In [13]:
vh = vcf.Reader(open('sample.vcf'))
ofh = open('lumpy_dels.vcf', 'w')
o = vcf.Writer(ofh, template=vh)
for rec in vh:
    if rec.INFO['SVTYPE'] == 'DEL' and 50 <= abs(rec.INFO['SVLEN'][0]) <= 30000:
        o.write_record(rec)
o.close()
ofh.close()

In [14]:
# repeat SURVIVOR analysis with lumpy data
!printf 'Personalis.hs37d5.filtered.vcf\nlumpy_dels.vcf\n' > fofn_tmp
!SURVIVOR merge fofn_tmp 100 1 1 1 0 50 svclassify_lumpy_merged.vcf
!rm fofn_tmp

merging entries: 2294
merging entries: 19307


In [15]:
# load the results
results = []
for l in open('svclassify_lumpy_merged.vcf'):
    if not l.startswith('#'):
        l = l.rstrip().split('\t')
        d = dict(x.split('=') for x in l[7].split(';') if x)
        truth_supp, lr_supp = map(bool, map(int, list(d['SUPP_VEC'])))
        results.append([l[0], l[1], truth_supp, lr_supp, d['AVGLEN']])
survivor_df = pd.DataFrame(results, columns=['chrom', 'pos', 'truth_supp', 'lr_supp', 'svlen'])
both = map(int, map(round, map(float, list(survivor_df[(survivor_df.truth_supp == True) & (survivor_df.lr_supp == True)].svlen))))
truth_only = map(int, map(round, map(float, list(survivor_df[(survivor_df.truth_supp == True) & (survivor_df.lr_supp == False)].svlen))))
lr_only = map(int, map(round, map(float, list(survivor_df[(survivor_df.truth_supp == False) & (survivor_df.lr_supp == True)].svlen))))
print len(both), len(truth_only), len(lr_only)

# construct a triplet of plots showing 50-500bp and 500-10000
with open('lumpy_truth_svclassify_size_distributions_personalis.pdf', 'w') as outf, PdfPages(outf) as pdf:
    fig, axes = plt.subplots(ncols=2)
    # 50-500bp
    ax = axes[0]
    ax.hist([[x for x in both if x <= 500], 
             [x for x in lr_only if x <= 500], 
             [x for x in truth_only if x <= 500]], 
            90, histtype='stepfilled', 
        label=['Clustered', 'Lumpy', 'Svclassify'], alpha=0.4)
    ax.set_xlabel('Size of deletion')
    ax.set_ylabel('Number of deletion calls')
    ax.set_title('50-500bp')
    ax.set_xlim((50, 500))
    # 500-5000bp
    ax = axes[1]
    ax.hist([[x for x in both if 500 <= x <= 10000], 
             [x for x in lr_only if 500 <= x <= 10000], 
             [x for x in truth_only if 500 <= x <= 10000]], 
            90, histtype='stepfilled', 
        label=['Clustered', 'Lumpy', 'Svclassify'], alpha=0.4)
    ax.set_xlabel('Size of deletion')
    ax.set_ylabel('Number of deletion calls')
    ax.set_title('500-10000bp')
    ax.set_xlim((500, 10000))
    ax.legend()
    fig.suptitle('Size distributions of SURVIVOR clustering of Lumpy deletions with Svclassify truth set')
    close_plot(pdf)

1263 1018 8307


In [20]:
filt = list(vcf.Reader(open('lumpy_dels.vcf')))
import numpy as np
np.mean([abs(x.INFO['SVLEN'][0]) for x in filt])

766.8432174858859

In [2]:
# repeat for un-merged dels
# large SV file is empty
!printf 'Personalis.hs37d5.filtered.vcf\ndels.filt.vcf\n' > fofn_tmp
!SURVIVOR merge fofn_tmp 100 1 1 1 0 50 svclassify_merged_dels_lr_only_personalis.vcf
!rm fofn_tmp


# load the results
results = []
for l in open('svclassify_merged_dels_lr_only_personalis.vcf'):
    if not l.startswith('#'):
        l = l.rstrip().split('\t')
        d = dict(x.split('=') for x in l[7].split(';') if x)
        truth_supp, lr_supp = map(bool, map(int, list(d['SUPP_VEC'])))
        results.append([l[0], l[1], truth_supp, lr_supp, d['AVGLEN']])
survivor_df = pd.DataFrame(results, columns=['chrom', 'pos', 'truth_supp', 'lr_supp', 'svlen'])
both = map(int, map(round, map(float, list(survivor_df[(survivor_df.truth_supp == True) & (survivor_df.lr_supp == True)].svlen))))
truth_only = map(int, map(round, map(float, list(survivor_df[(survivor_df.truth_supp == True) & (survivor_df.lr_supp == False)].svlen))))
lr_only = map(int, map(round, map(float, list(survivor_df[(survivor_df.truth_supp == False) & (survivor_df.lr_supp == True)].svlen))))
print len(both), len(truth_only), len(lr_only)

# construct a triplet of plots showing 50-500bp and 500-10000
with open('lr_truth_dels_only_svclassify_size_distributions_personalis.pdf', 'w') as outf, PdfPages(outf) as pdf:
    fig, axes = plt.subplots(ncols=2)
    # 50-500bp
    ax = axes[0]
    ax.hist([[x for x in both if x <= 500], 
             [x for x in lr_only if x <= 500], 
             [x for x in truth_only if x <= 500]], 
            90, histtype='stepfilled', 
        label=['Clustered', 'LongRanger', 'Svclassify'], alpha=0.4)
    ax.set_xlabel('Size of deletion')
    ax.set_ylabel('Number of deletion calls')
    ax.set_title('50-500bp')
    ax.set_xlim((50, 500))
    # 500-5000bp
    ax = axes[1]
    ax.hist([[x for x in both if 500 <= x <= 10000], 
             [x for x in lr_only if 500 <= x <= 10000], 
             [x for x in truth_only if 500 <= x <= 10000]], 
            90, histtype='stepfilled', 
        label=['Clustered', 'LongRanger', 'Svclassify'], alpha=0.4)
    ax.set_xlabel('Size of deletion')
    ax.set_ylabel('Number of deletion calls')
    ax.set_title('500-10000bp')
    ax.set_xlim((500, 10000))
    ax.legend()
    fig.suptitle('Size distributions of SURVIVOR clustering of LongRanger deletions with Svclassify truth set')
    close_plot(pdf)

merging entries: 2294
merging entries: 4118
2017 264 2048


In [5]:
recs = [x.split() for x in open('svclassify_merged_dels_lr_only_personalis.vcf') if not x.startswith('#')]
hom = [x for x in recs if x[-1].startswith('1/1') or x[-1].startswith('1|1')]
print len(hom), len(recs) - len(hom)

1699 2630
